In [ ]:
import os
import pandas as pd

In [ ]:
filepath_old = '/media/sf_VBox_Shared/Arabic/Fiqh/2018-04-24-Fiqh/Fiqh'
filepath_new = '/media/sf_VBox_Shared/Arabic/Fiqh/2018-06-08-Fiqh/'

In [ ]:

def get_metadata(filepath):
    metadata_dict = {}
    
    for filename in os.listdir(filepath):
        with open(os.path.join(filepath, filename)) as f:
            metadata = {}
            for line in f.readlines():
                # TODO: metadata is sometimes inconsistent, (missing # before META,
                # and fields not separated by :: but single :)
                if line.startswith('#META#'):
                    splitted = line.split(u'::')
                    if(len(splitted)==2):
                        name, value = line.split(u'::')

                        value = value.strip()
                        name = name.strip()

                        # only save metadata that has a value
                        #if value != 'NODATA':
                        _, name = name.split(u' ', 1)
                        name = name.replace(u' ', u'_')
                        # remove left to right mark
                        name = name.replace(u"\u200F", u'')
                        name = name.split(u'.')[-1]
                        metadata[name] = value
            metadata_dict[filename] = metadata
    return metadata_dict

In [ ]:
metadata_old = get_metadata(filepath_old)

In [ ]:
metadata_new = get_metadata(filepath_new)

In [ ]:
metadata_old_df = pd.DataFrame.from_dict(metadata_old, orient='index')
metadata_new_df = pd.DataFrame.from_dict(metadata_new, orient='index')

In [ ]:
metadata_old_df.index.name = 'filename'
metadata_new_df.index.name = 'filename'

In [ ]:
metadata_df.to_csv(os.path.join(filepath, 'metadata-from-files.csv'))

In [ ]:
metadata_old_df.info()

In [ ]:
metadata_new_df.info()

In [ ]:
metadata_old_df.head()

In [ ]:
metadata_new_df.head()

In [ ]:
for col in metadata_old_df.columns:
    if col in metadata_new_df.columns:
        print(col, len(set(metadata_old_df[col]).intersection(set(metadata_new_df[col]))))

In [ ]:
metadata_old_df.LibURI.value_counts().head()

In [ ]:
metadata_new_df.LibURI.value_counts().head()

In [ ]:
metadata_old_df.SortField.value_counts().head(), metadata_new_df.SortField.value_counts().head()

In [ ]:
metadata_old_df.groupby(['SortField', 'BookTITLE']).size().sort_values(ascending=False).head()

In [ ]:
title_to_filename = metadata_old_df[['SortField', 'BookTITLE']].reset_index()

In [ ]:
print(len(title_to_filename))
print(title_to_filename[['SortField', 'BookTITLE']].drop_duplicates().shape)

In [ ]:
merged = metadata_new_df.reset_index().merge(title_to_filename, left_on=['SortField', 'BookTITLE'], right_on=['SortField', 'BookTITLE'], how='left', suffixes=('_new', '_old'))

In [ ]:
merged['filename_old']

In [ ]:
merged.filename_old.value_counts().head()

In [ ]:
merged[merged.filename_old.isnull()]

In [ ]:
merged = merged.dropna(subset=['filename_old'])

In [ ]:
merged.to_csv('/media/sf_VBox_Shared/Arabic/Fiqh/merged_metadata.csv', index=False)